In [1]:
import jutil
import numpy as np
from collections import Counter
import pyfpgrowth

In [2]:
sql = "select t.Course_code as course_code from train_plan_course t,train_plan_credit c where t.Course_big_type_id=c.sid \
and t.Grade='2014' and c.type_name like '%专业%'  and t.Speciality_code='0408'"
code = [d['course_code'] for d in jutil.load_data(sql)]

In [3]:
len(set(code))

53

In [4]:
sql = "select student_id,course_name,course_code,pmark,mark,term_order from view_stu_course_mark where speciality_code='0408' and grade='2014'"
df = jutil.load_pd_df(sql)
df.head()

course_code course_name mark  pmark student_id  term_order
0     0471108        操作系统    7   81.0   20122639           3
1     0474003    计算机图形学实验    9   85.0   20122639           4
2     0474012      操作系统实验    9   88.0   20122639           3
3     0474014  Java程序设计实验   10   95.0   20122639           3
4     0474019    人机交互技术实验    8   83.0   20122639           4

In [5]:
# 选出了与专业相关的课程
dd = df[df['course_code'].isin(code)]

## 按学生group,然后将其课程分数作为维度,学生数作为数据量

In [6]:
code = dd.drop_duplicates(subset=['student_id','course_code']).groupby(['student_id'],as_index=False)['course_code'].apply(list)
mark = dd.drop_duplicates(subset=['student_id','course_code']).groupby(['student_id'],as_index=False)['mark'].apply(list)
m = dd.drop_duplicates(subset=['student_id','course_code']).groupby(['student_id'],as_index=False)['pmark'].apply(list)
md = m.to_dict()
marks = list(m)

In [7]:
# 得到大多数同学都有的课程
cnt = [len(m) for m in marks]
most_len = max(set(cnt),key=cnt.count)
# most_len
cc = dd.drop_duplicates(subset=['student_id','course_code']).groupby(['student_id'],as_index=False)['course_name'].apply(list)
course_names = []
course_codes = []
for k,v in cc.iteritems():
    if len(v)==most_len:
        course_names = v
        break
for k,v in code.iteritems():
    if len(v)==most_len:
        course_codes = v
        break
course_names

['数据库技术与应用实验',
 '数字电子技术B',
 '高级语言程序设计',
 '微机与接口技术实验',
 '软件工程概论',
 '软件系统分析与设计',
 '数据结构实验',
 '计算机导论A',
 '算法设计与分析',
 '大学物理实验Ⅰ',
 '软件设计实习',
 '人机交互技术',
 '人机交互技术实验',
 '软件项目管理',
 '高等数学Ⅰ',
 '高等数学Ⅱ',
 '数学建模B',
 '离散数学A',
 '软件工程/网络软件开发实习',
 'J2EE体系结构及程序设计',
 '新技术讲座',
 '数据结构A',
 '计算机图形学',
 '计算机网络工程实验',
 '数字电子技术实验B(电类)',
 '面向对象程序设计实验',
 '计算机组成原理B',
 '数据库技术与应用',
 '计算机图形学实验',
 '操作系统实验',
 '计算机组成实验C',
 '面向对象程序设计',
 '程序语言综合课程设计',
 '数据库原理',
 '数据库原理实验',
 '微机与接口技术A',
 'Java程序设计',
 '计算方法C',
 'Java程序设计实验',
 '高级语言程序设计实验',
 '软件质量保证与测试',
 '线性代数B',
 '大学物理实验Ⅱ']

In [8]:
gd = dd.drop_duplicates(subset=['student_id','course_code']).groupby(['student_id'],as_index=False)
d_arr = dict() # key存的是课程名,value是存放成绩的数组,这样竖着看就是一个学生的成绩列表了,也就是转置操作
d_arr2 = dict() 
for name ,g in gd:
    d_m = dict()
    d_m2 = dict()
    for i,row in g.iterrows():
        d_m[row['course_name']] = row['pmark']
        d_m2[row['course_name']] = row['mark']
    for c in course_names:
        d_arr[c] = d_arr.get(c,[])
        d_arr[c].append(d_m.get(c,60))
        d_arr2[c] = d_arr2.get(c,[])
        d_arr2[c].append(d_m2.get(c,1))
courses = d_arr.keys()
mt_old = [arr for arr in d_arr.values()]
mt2_old = [arr for arr in d_arr2.values()]
mt = np.transpose(mt_old)
mt

array([[88. , 98. , 96. , ..., 77. , 94. , 80.7],
       [79. , 78. , 74. , ..., 73. , 60. ,  5.6],
       [60. , 46. , 86. , ..., 52. , 70. , 75.7],
       ...,
       [60. , 97. , 60. , ..., 78. , 74. , 83. ],
       [81. , 89. , 94. , ..., 71. , 75. , 76.2],
       [90. , 74. , 80. , ..., 82. , 71. , 60. ]])

In [9]:
# 利用mt进行特征选择,选取最优的20门课
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import f_regression

In [27]:
sel = VarianceThreshold(threshold=120)
sel.fit_transform(mt).shape

(120, 14)

In [9]:
indices = sel.get_support(indices=True)
cccc = []
mt_new = []
mt_new2 = []
course_select = []
for i in indices:
    cccc.append(course_codes[i])
    course_select.append(course_names[i])
    mt_new.append(mt_old[i])
    mt_new2.append(mt2_old[i])
cccc
course_select

NameError: name 'sel' is not defined

In [29]:
mt_new = np.transpose(mt_new)
mt_new

array([[94., 92., 84., ..., 98., 96., 96.],
       [61.,  8., 33., ..., 78., 74., 69.],
       [71., 77., 74., ...,  5., 86., 15.],
       ...,
       [73., 73., 78., ..., 97., 60., 60.],
       [86., 80., 79., ..., 89., 94., 60.],
       [67., 86., 80., ..., 74., 80., 88.]])

In [10]:
mt_new2 = np.transpose(mt_new2)
mt_new2

NameError: name 'mt_new2' is not defined

In [31]:
transac = []
for row in mt_new2:
    t = []
    for i in range(len(cccc)):
        if str(row[i])>'8':
            t.append(cccc[i]+'_2')
        elif str(row[i])>'4':
            t.append(cccc[i]+'_1')
        else:
            t.append(cccc[i]+'_0')
    transac.append(t)
transac

[['6011310_0',
  '6011320_0',
  '6033100_1',
  '3100312_1',
  '6010500_0',
  '0471071_2',
  '3243020_1',
  '6041943_1',
  '0471075_2',
  '3244152_1',
  '3232100_0',
  '3045600_0',
  '3243781_0',
  '3243120_0'],
 ['6011310_0',
  '6011320_0',
  '6033100_0',
  '3100312_0',
  '6010500_0',
  '0471071_0',
  '3243020_0',
  '6041943_0',
  '0471075_0',
  '3244152_0',
  '3232100_0',
  '3045600_1',
  '3243781_1',
  '3243120_0'],
 ['6011310_0',
  '6011320_1',
  '6033100_1',
  '3100312_1',
  '6010500_0',
  '0471071_0',
  '3243020_0',
  '6041943_0',
  '0471075_0',
  '3244152_0',
  '3232100_0',
  '3045600_0',
  '3243781_2',
  '3243120_0'],
 ['6011310_0',
  '6011320_0',
  '6033100_0',
  '3100312_1',
  '6010500_1',
  '0471071_0',
  '3243020_0',
  '6041943_0',
  '0471075_0',
  '3244152_0',
  '3232100_0',
  '3045600_0',
  '3243781_0',
  '3243120_0'],
 ['6011310_2',
  '6011320_1',
  '6033100_0',
  '3100312_0',
  '6010500_1',
  '0471071_0',
  '3243020_0',
  '6041943_0',
  '0471075_0',
  '3244152_0',
  '323

In [15]:
from pymining import itemmining,assocrules

In [48]:
relim_input = itemmining.get_relim_input(transac)

In [49]:
item_sets = itemmining.relim(relim_input, min_support=8)
item_sets

{frozenset({'0471075_2'}): 9,
 frozenset({'0471075_2', '3243020_1'}): 8,
 frozenset({'0471075_2', '3045600_0', '3243020_1'}): 8,
 frozenset({'0471075_2', '3045600_0'}): 9,
 frozenset({'6033100_2'}): 10,
 frozenset({'3045600_0', '6033100_2'}): 8,
 frozenset({'3100312_2'}): 11,
 frozenset({'3100312_2', '6011310_0'}): 9,
 frozenset({'3045600_0', '3100312_2'}): 8,
 frozenset({'0471071_2'}): 14,
 frozenset({'0471071_2', '3243020_1'}): 8,
 frozenset({'0471071_2', '3045600_0'}): 9,
 frozenset({'0471071_2', '6041943_0'}): 11,
 frozenset({'0471071_2', '3244152_0', '6041943_0'}): 8,
 frozenset({'0471071_2', '3244152_0'}): 8,
 frozenset({'6011320_2'}): 16,
 frozenset({'3243020_1', '6011320_2'}): 12,
 frozenset({'3243020_1', '6011310_0', '6011320_2'}): 9,
 frozenset({'3045600_0', '3243020_1', '6011320_2'}): 9,
 frozenset({'3243781_0', '6011320_2'}): 8,
 frozenset({'0471071_0', '6011320_2'}): 8,
 frozenset({'6011310_0', '6011320_2'}): 11,
 frozenset({'3045600_0', '6011310_0', '6011320_2'}): 8,
 fro

In [50]:
ules = assocrules.mine_assoc_rules(item_sets, min_support=8, min_confidence=0.7)
ules

[(frozenset({'0471075_0',
             '3045600_0',
             '3100312_0',
             '3243020_0',
             '3243781_0',
             '3244152_0',
             '6010500_0',
             '6011310_0',
             '6011320_0',
             '6033100_0',
             '6041943_0'}),
  frozenset({'3232100_0'}),
  8,
  1.0),
 (frozenset({'3045600_0',
             '3100312_0',
             '3243020_0',
             '3243781_0',
             '3244152_0',
             '6010500_0',
             '6011310_0',
             '6011320_0',
             '6033100_0',
             '6041943_0'}),
  frozenset({'0471075_0', '3232100_0'}),
  8,
  1.0),
 (frozenset({'3045600_0',
             '3100312_0',
             '3243781_0',
             '3244152_0',
             '6010500_0',
             '6011310_0',
             '6011320_0',
             '6033100_0',
             '6041943_0'}),
  frozenset({'0471075_0', '3232100_0', '3243020_0'}),
  8,
  0.8888888888888888),
 (frozenset({'3045600_0',
           